# advent of code 2024 - [day 5](https://adventofcode.com/2024/day/5)

In [107]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']


In [108]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

import pandas as pd

2.13.0


In [109]:
queries = [
"CALL apoc.schema.assert({},{});",
"""MATCH (n)
CALL {WITH n DETACH DELETE n}
IN TRANSACTIONS OF 1000 ROWS;""",
"""CALL gds.graph.list()
YIELD graphName
WITH graphName AS g
CALL (g) {CALL gds.graph.drop(g) YIELD graphName RETURN graphName}
WITH graphName RETURN graphName;"""]

for q in queries:
    gds.run_cypher(q, {})


## Neo4j-based solution

In [110]:
file = "input.txt"
#file = "test.txt"

### Parsing

In [111]:
def gen_rules(file='input.txt'):
    """Generates tuples of integers"""
    file = open(file, 'r')
    for _, line in enumerate(file):
        if line.strip() == "":
            break
        els = line.strip().split("|")
        els = [int(el) for el in els]
        yield {'before': els[0], 'after': els[1]}

In [112]:
def gen_update(file='input.txt'):
    """Generates tuples of integers"""
    file = open(file, 'r')
    for _, line in enumerate(file):
        if '|' in line or line.strip() == "":
            continue
        els = line.strip().split(",")
        els = [int(el) for el in els]
        yield els

### Ingestion

In [113]:
gds.run_cypher('CREATE INDEX page_number IF NOT EXISTS FOR (p:Page) ON (p.number)')

""


In [114]:
rules = list(gen_rules(file))


In [115]:
query_ingest = """
UNWIND $rules AS rule
MERGE (b:Page {number:rule.before})
MERGE (a:Page {number:rule.after})
MERGE (b)-[:BEFORE]->(a)
"""

gds.run_cypher(query_ingest, {"rules":rules})

""


In [116]:
updates = list(gen_update(file))

In [117]:
query_ingest_updates = """
UNWIND $updates AS update
CREATE (u:Update {page_list: update})
"""
gds.run_cypher(query_ingest_updates, {"updates":updates})

""


In [118]:
gds.run_cypher('CREATE INDEX comp_before_after IF NOT EXISTS FOR (c:Comp) ON (c.before, c.after)')
gds.run_cypher('CREATE INDEX update_page_list IF NOT EXISTS FOR (u:Update) ON (u.page_list)')

""


In [119]:
gds.run_cypher("""MATCH (u:Update)
WITH u, u.page_list AS pl
UNWIND range(0, size(pl)-2) AS ix
WITH u, pl, pl[ix] AS before, pl[ix+1] AS after
MERGE (comp:Comp {before:before, after:after})
MERGE (u)-[:REQUIRE]->(comp)
MERGE (before_page:Page {number:before})
MERGE (after_page:Page {number:after})
MERGE (comp)-[:COMES_BEFORE]->(before_page)
MERGE (comp)-[:COMES_AFTER]->(after_page)
WITH u, pl[size(pl)/2] AS median
MERGE (median_page:Page {number:median})
MERGE (u)-[:MEDIAN]->(median_page)""", {})

""


In [120]:
gds.run_cypher("""MATCH path = (bef)<-[:COMES_BEFORE]-(comp:Comp)-[:COMES_AFTER]->(aft) WHERE EXISTS {(aft)-[:BEFORE]->(bef)}
SET comp:NOK""", {})

""


In [121]:
gds.run_cypher("""MATCH (u:Update)-[:MEDIAN]->(p) WHERE NOT EXISTS {(u)-[:REQUIRE]->(:NOK)}
RETURN sum (p.number) AS part1""", {})


,part1
0,4996


In [138]:
gds.run_cypher("""MATCH (u:Update)-[:REQUIRE]->(:NOK)
WITH DISTINCT u
CALL (u) {
WITH u, [(u)-[:REQUIRE]->()-->(page)| page] AS pages
UNWIND pages AS source
WITH DISTINCT u, source, pages
OPTIONAL MATCH (source)-[r:BEFORE]->(target WHERE target IN pages)
WITH u, gds.graph.project('dag_'+elementId(u), source, target) AS g
WITH u, g.graphName AS graph, g.nodeCount AS nodes, g.relationshipCount AS rels
CALL gds.dag.topologicalSort.stream(graph)
YIELD nodeId
WITH u, nodeId
WITH u, collect(nodeId) AS ordered_nodes
WITH u, ordered_nodes[size(ordered_nodes)/2] AS median
WITH u, median
CALL gds.graph.drop('dag_'+elementId(u)) YIELD graphName
RETURN gds.util.asNode(median).number AS median} IN CONCURRENT TRANSACTIONS OF 10 ROWS
RETURN sum(median) AS part2""", {})

,part2
0,6311
